<a href="https://colab.research.google.com/github/Yutitam/grounding/blob/main/GND_CVXPY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Just in case the code couldn't run na kub
!pip install cylp
!pip install cvxopt
!pip install pulp
!pip install "cvxpy[CBC]"
!pip install cvxpy numpy
!pip install ecos # For ECOS solver
!pip install scs  # For SCS solver

In [ ]:
import cvxpy as cp
import numpy as np

# --- Design Input Parameters (from your tables / typical values) ---
Ig = 63000          # Total fault current (A)
t_fault = 1         # Maximum fault duration (s)
ambient_temp = 40   # Ambient temperature (C) (not directly used in simplified V formulas here, but relevant for conductor sizing)
X_R_ratio = 20      # X_0/R_0 ratio (not directly used in this simplified model)
system_voltage = 230 # System Voltage (kV) (not directly used in this model)
rho_soil = 16.804   # Soil resistivity (ohm-m)
rho_crushed_rock_wet = 2500 # Crushed-rock resistivity (wet) (ohm-m)
thickness_crushed_rock = 0.1 # Thickness of crushed-rock surface (m)
ground_grid_depth = 0.5 # Ground grid depth, below rough grade (m) -> This is 'h' in the Rg formula
available_area_L = 100 # Available grounding area Length (m)
available_area_W = 100 # Available grounding area Width (m)

# --- Conductor and Rod Parameters ---
conductor_diameter = 0.0127
fixed_rod_length = 7.5
body_fibrillation_constant = 0.116

# --- Cost Parameters ---
Kc = 2150           # Cost per meter of conductor (THB/m)
Kr = 2560           # Cost per rod (THB) - simplified, assuming installation included

print("--- Grounding Grid Optimization (Convex Approximation based on IEEE 80 Concepts) ---")
print(f"Fault Current (Ig): {Ig} A")
print(f"Fault Duration (t_fault): {t_fault} s")
print(f"Soil Resistivity (rho_soil): {rho_soil} ohm-m")
print(f"Crushed Rock Resistivity (rho_crushed_rock_wet): {rho_crushed_rock_wet} ohm-m")
print(f"Crushed Rock Thickness: {thickness_crushed_rock} m")
print(f"Ground Grid Depth (h): {ground_grid_depth} m")
print(f"Available Area: {available_area_L}m x {available_area_W}m")
print(f"Fixed Rod Length: {fixed_rod_length} m")
print(f"Conductor Diameter: {conductor_diameter*1000:.2f} mm")


# --- Derived Parameters (Permissible Step/Touch Voltages based on IEEE 80) ---

try:
    if rho_crushed_rock_wet == 0 or rho_soil == 0:
        Cs = 1.0
    else:
        Cs = 1 - (0.096 * (1 - (rho_soil / rho_crushed_rock_wet)))
except ZeroDivisionError:
    Cs = 1.0

# Permissible Touch Voltage (Etouch_perm) - IEEE 80 Eq. 37 (for 50kg person)
E_touch_perm = (1000 + (1.5 * Cs * rho_crushed_rock_wet)) * (body_fibrillation_constant / np.sqrt(t_fault))

# Permissible Step Voltage (Estep_perm) - IEEE 80 Eq. 40 (for 50kg person)
E_step_perm = (1000 + (6 * Cs * rho_crushed_rock_wet)) * (body_fibrillation_constant / np.sqrt(t_fault))

print(f"\nCalculated Permissible Touch Voltage (E_touch_perm): {E_touch_perm:.2f} V")
print(f"Calculated Permissible Step Voltage (E_step_perm): {E_step_perm:.2f} V")

# --- Decision Variables (Continuous relaxation for easier solving) ---
# We will round these up after solving
num_mesh_rows = cp.Variable(pos=True) # Number of parallel conductors along grid width
num_mesh_cols = cp.Variable(pos=True) # Number of parallel conductors along grid length
num_rods = cp.Variable(pos=True) # Number of ground rods
num_rods_quarter = cp.Variable(pos=True) # Helper variable for num_rods to be divisible by 4

# Assuming grid covers available area
grid_L = available_area_L
grid_W = available_area_W
grid_Area = grid_L * grid_W


#L_c - Total length of grid conductors
total_conductor_length = num_mesh_rows * grid_W + num_mesh_cols * grid_L

#L_r - Total length of ground rods
total_rod_length = num_rods * fixed_rod_length

#L_T - Total length of buried conductors (grid + rods)
L_T = total_conductor_length + total_rod_length

# --- Grid Resistance (Rg) - Using the provided formula (from image) ---
# Rg = ρ [ 1/LT + 1/sqrt(20A) * (1 + 1 / (1 + h*sqrt(20/A))) ]

constant_sqrt_20A = np.sqrt(20 * grid_Area)
constant_h_sqrt_20_over_A = ground_grid_depth * np.sqrt(20 / grid_Area)

# Rg expression using CVXPY's inv_pos for 1/LT to maintain convexity
# L_T must be positive for cp.inv_pos
Rg = rho_soil * ( cp.inv_pos(L_T) + (1 / constant_sqrt_20A) * (1 + (1 / (1 + constant_h_sqrt_20_over_A))) )

alpha_mesh_factor = 0.478*3.752 # Represents a combined K_m * K_i * (some constant related to geometry)
alpha_step_factor = 0.44*3.752 # Represents a combined K_s * K_i * (some constant related to geometry)

V_mesh_achieved = Ig * rho_soil * alpha_mesh_factor * cp.inv_pos(L_T)
V_step_achieved = Ig * rho_soil * alpha_step_factor * cp.inv_pos(L_T)

# --- Objective Function ---
cost = total_conductor_length * Kc + num_rods * Kr
objective = cp.Minimize(cost)

# --- Constraints ---
constraints = [
    # Safety Constraints
    V_mesh_achieved <= E_touch_perm,
    V_step_achieved <= E_step_perm,

    # Grid Geometry Constraints
    num_rods >= 1, # At least one rod
    num_mesh_rows >= 2, # At least 2 parallel conductors in each direction for a practical mesh
    num_mesh_cols >= 2,

    # Total conductor length minimum for a practical grid
    total_conductor_length >= (available_area_L + available_area_W) * 2, # Minimum for perimeter + internal runs

    # Ensure total length is within reasonable bounds (e.g., cannot exceed an absurd density)
    total_conductor_length <= 2 * (available_area_L * 100 + available_area_W * 100), # Max total length, adjusted up

    # Ensure num_rods is divisible by 4 and at least 4 (for continuous variables, this ensures optimal is a multiple)
    num_rods == 4 * num_rods_quarter, # num_rods must be a multiple of 4
    num_rods_quarter >= 1,            # Ensures num_rods is at least 4 (4 * 1)

    # Ensure square grounding design
    num_mesh_rows == num_mesh_cols,

    # Ensure L_T is positive for cp.inv_pos
    L_T >= 0.1 # Small positive lower bound
]

# --- Solve ---

problem = cp.Problem(objective, constraints)
# CVXPY will automatically select an appropriate installed solver
# (like SCS or ECOS or Clarabel) that can handle Conic Programs.
problem.solve(verbose=False)


# --- Output ---
print("\n--- Optimization Results ---")
if problem.status in [cp.OPTIMAL, cp.OPTIMAL_INACCURATE]:
    print(f"Problem Status: {problem.status}")
    print(f"Optimal Continuous Cost: {problem.value:.2f} THB (before rounding)")

    # --- Post-processing: Round up continuous variables to integers ---
    num_rods_final = int(np.ceil(num_rods.value)) if num_rods.value is not None else 0
    num_mesh_rows_final = int(np.ceil(num_mesh_rows.value)) if num_mesh_rows.value is not None else 0
    num_mesh_cols_final = int(np.ceil(num_mesh_cols.value)) if num_mesh_cols.value is not None else 0

    # Ensure num_rods_final is still a multiple of 4 after rounding up
    # This might make it slightly higher than strictly necessary if original value was e.g., 4.1 -> 8
    if num_rods_final % 4 != 0:
        num_rods_final = int(np.ceil(num_rods_final / 4.0)) * 4
    # Ensure minimums after rounding, if rounding down somehow occurred or initial was very small
    num_rods_final = max(num_rods_final, 4) # Minimum 4 rods
    num_mesh_rows_final = max(num_mesh_rows_final, 2)
    num_mesh_cols_final = max(num_mesh_cols_final, 2)


    # --- Recalculate based on rounded integer values ---
    total_conductor_length_final = num_mesh_rows_final * grid_W + num_mesh_cols_final * grid_L
    total_rod_length_final = num_rods_final * fixed_rod_length
    L_T_final = total_conductor_length_final + total_rod_length_final
    cost_final = total_conductor_length_final * Kc + num_rods_final * Kr

    print(f"\n--- Practical Integer Design (Rounded Up) ---")
    print(f"Final Cost: {cost_final:.2f} THB")
    print(f"Number of Rods: {num_rods_final}")
    print(f"Total Rod Length: {total_rod_length_final:.2f} m")
    print(f"Optimal Number of Mesh Rows: {num_mesh_rows_final}")
    print(f"Optimal Number of Mesh Columns: {num_mesh_cols_final}")
    print(f"Total Conductor Length (Lc): {total_conductor_length_final:.2f} m")
    print(f"Total Buried Length (LT = Lc + Lr): {L_T_final:.2f} m")


    # Recalculate Rg and GPR based on FINAL (rounded) L_T
    if L_T_final > 0:
        Rg_display_final = rho_soil * ( (1 / L_T_final) + (1 / constant_sqrt_20A) * (1 + (1 / (1 + constant_h_sqrt_20_over_A))) )
    else:
        Rg_display_final = float('inf')

    GPR_display_final = Ig * Rg_display_final

    print(f"Estimated Grid Resistance (Rg): {Rg_display_final:.4f} ohms")
    print(f"Estimated Ground Potential Rise (GPR = Ig * Rg): {GPR_display_final:.2f} V")

    # Display achieved voltages based on the FINAL (rounded) L_T
    if L_T_final > 0:
        achieved_mesh_final = Ig * rho_soil * alpha_mesh_factor * (1 / L_T_final)
        achieved_step_final = Ig * rho_soil * alpha_step_factor * (1 / L_T_final)
    else:
        achieved_mesh_final = float('inf')
        achieved_step_final = float('inf')

    print(f"Achieved Touch Voltage: {achieved_mesh_final:.2f} V (Permitted: {E_touch_perm:.2f} V)")
    print(f"Achieved Step Voltage: {achieved_step_final:.2f} V (Permitted: {E_step_perm:.2f} V)")

    # Check if safety constraints are still met with rounded values (should be, as we rounded up material)
    if achieved_mesh_final <= E_touch_perm and achieved_step_final <= E_step_perm:
        print("Safety requirements remain satisfied after rounding up.")
    else:
        print("WARNING: Safety requirements might be violated after rounding up. Re-evaluate design.")


    # Calculate approximate conductor spacing
    if num_mesh_rows_final > 1 and num_mesh_cols_final > 1:
        avg_spacing_L = grid_L / (num_mesh_cols_final - 1)
        avg_spacing_W = grid_W / (num_mesh_rows_final - 1)
        print(f"Estimated Avg. Conductor Spacing (L-dir): {avg_spacing_L:.2f} m")
        print(f"Estimated Avg. Conductor Spacing (W-dir): {avg_spacing_W:.2f} m")
    else:
        print("Cannot estimate spacing accurately with current mesh configuration (too few conductors).")


elif problem.status == cp.INFEASIBLE:
    print("Problem is infeasible. No solution found that satisfies all constraints in the continuous relaxation.")
    print("Consider relaxing constraints (e.g., min length, max voltage) or checking input parameters.")
    print(f"Permissible Touch Voltage (E_touch_perm): {E_touch_perm:.2f} V")
    print(f"Permissible Step Voltage (E_step_perm): {E_step_perm:.2f} V")
elif problem.status == cp.UNBOUNDED:
    print("Problem is unbounded. The cost can be infinitely decreased.")
    print("Check objective function and constraints for missing lower bounds.")
else:
    print(f"Problem could not be solved. Status: {problem.status}")
    print("An error occurred during solving, or the solver did not converge. Check solver output if verbose=True.")